In [1]:
!conda install -c conda-forge beautifulsoup4 --yes # web scraping
!conda install -c conda-forge ProgressBar2 --yes
!conda install -c conda-forge lxml --yes # parser for html web scraping
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge html5lib --yes # parser for html5 for beautifulsoup

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    beautifulsoup4-4.6.3       |           py35_0         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0        conda-forge
    ca-certificates: 2019.1.23-0          --> 2019.3.9-hecc5488_0 conda-forge
    certifi:         2018.8.24-py35_1     --> 2018.8.24-py35_1001 

In [229]:
    from progressbar import ProgressBar
    from bs4 import BeautifulSoup as bts # web scraping
    import numpy as np # library to handle data in a vectorized manner
    import pandas as pd # library for data analysis
    from pandas.io.json import json_normalize # json data
    import matplotlib.cm as cm
    import matplotlib.colors as colors
    import requests # library to handle requests
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import matplotlib as mp # library for visualization
    from sklearn.cluster import KMeans # import k-means from clustering stage
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import folium # map rendering library
    import lxml 
    import re
    from time import sleep
    print("Libraries imported successfully!\n")

Libraries imported successfully!



In [230]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [231]:
source = requests.get(url).text

In [232]:
filename = "canada1.cvs"
f = open(filename, 'w')

In [233]:
soup = bts(source, 'lxml')

In [234]:
article = soup.find('div', 'mw-content-ltr')

In [235]:
table = article.table

In [268]:
rows = table.tbody.find_all('tr')

In [269]:
column_name = rowt[0].find_all('th')
print(column_name)

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
</th>]


In [270]:
headers = ""

In [271]:
for column_name in column_names:
    headers = headers + column_name.text + ","
headers = headers[:-1]

In [272]:
f.write(headers)

31

In [273]:
for row in rows:
    cells = row.find_all('td')
    row_buff = ""
    for cell in cells:
        row_buff = row_buff + cell.text + ','

    # A comma is not required at the end of each line/row
    row_buff = row_buff[:-1]
    f.write(row_buff)

In [274]:
df = pd.read_csv("canada1.cvs")
df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,Postcode,Borough,Neighbourhood
2,Postcode,Borough,Neighbourhood
3,M1A,Not assigned,Not assigned
4,M2A,Not assigned,Not assigned


In [275]:
df.shape

(543, 3)

In [278]:
borough_mask = df.index[df['Borough'] == 'Not assigned']
neighborhood_mask = df.index[df['Neighbourhood'] == 'Not assigned']
neighborhood_and_borough_mask = borough_mask & neighborhood_mask

In [288]:
print('  {} Postal codes'.format(df['Postcode'].unique().shape[0]))
print('  {} Boroughs'.format(df['Borough'].unique().shape[1]))
print('  {} Neighborhoods'.format(df['Neighbourhood'].unique().shape[0]))
print('The DataFrame shape is {}'.format(df.shape),'\n')

  181 Postal codes


IndexError: tuple index out of range

(0, 3)

In [127]:
# headline = article.a.text
# print(headline)

In [124]:
# print(article.prettify())

In [125]:
# summary = article.find('div', class_='entry-content')

In [126]:
# print(summary)

In [102]:
vid_src = article.find('iframe', class_='youtube-player')['src']
print(vid_src)

https://www.youtube.com/embed/-nh9rCzPJ20?version=3&rel=1&fs=1&autohide=2&showsearch=0&showinfo=1&iv_load_policy=1&wmode=transparent


In [111]:
vid_id = vid_src.split('/')[4]
vid_id = vid_id.split('?')[0]

In [112]:
# print(vid_id)

-nh9rCzPJ20
